# Significance of Eigenvalues and Eigenvectors in Corner Detection

The Harris corner detector uses the eigenvalues of an autocorrelation matrix to determine the presence of corners in an image. The following demonstration illustrates their significance.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create synthetic image with white square
image = np.zeros((100, 100), dtype=np.float64)
image[30:70, 30:70] = 1.0  # White square

# Compute gradients using finite differences
Ix = np.zeros_like(image)
Iy = np.zeros_like(image)
Ix[:, :-1] = image[:, 1:] - image[:, :-1]  # X-gradient
Iy[:-1, :] = image[1:, :] - image[:-1, :]  # Y-gradient

# Structure tensor components
Ix2, Iy2 = Ix**2, Iy**2
Ixy = Ix * Iy

def analyze_point(x, y, window_size=3):
    """Compute eigenvalues/vectors for a pixel neighborhood"""
    half = window_size//2
    M = np.array([
        [Ix2[y-half:y+half+1, x-half:x+half+1].sum(),
         Ixy[y-half:y+half+1, x-half:x+half+1].sum()],
        [Ixy[y-half:y+half+1, x-half:x+half+1].sum(),
         Iy2[y-half:y+half+1, x-half:x+half+1].sum()]
    ])
    return np.linalg.eig(M)

# Analyze corner (30,30) and flat area (10,10)
corner_vals, corner_vecs = analyze_point(30, 30)
flat_vals, flat_vecs = analyze_point(10, 10)
edge_vals, edge_vecs = analyze_point(50, 30)

print("Corner eigenvalues:", corner_vals)
print("Corner eigenvectors:\n", corner_vecs)
print("\nEdge eigenvalues:", edge_vals)
print("Edge eigenvectors:\n", edge_vecs)
print("\nFlat area eigenvalues:", flat_vals)
print("Flat area eigenvectors:\n", flat_vecs)


In [ ]:
plt.imshow(image, cmap='gray')
plt.scatter([30,10,50], [30,10,30], c=['r','b','g'], s=100)
plt.quiver(30,30, *corner_vecs[:,1]*20, color='red', scale=1)  # Max eigenvector
plt.quiver(50,30, *edge_vecs[:,1]*20, color='green', scale=1)  # Edge max eigenvector
plt.quiver(10,10, *flat_vecs[:,1]*20, color='blue', scale=1)
plt.title('Eigenvector Directions: Corner (red) vs Flat (blue) vs Edge (green)')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import rotate

# Create a black image with a white square
image_size = 100
square_size = 40
image = np.zeros((image_size, image_size), dtype=np.float64)
start = (image_size - square_size) // 2
end = start + square_size
image[start:end, start:end] = 1.0

# Rotate the image by 45 degrees around the center without changing shape
rotated_image = rotate(image, angle=45, reshape=False, order=1)

# Compute gradients using simple finite differences
Ix = np.zeros_like(rotated_image)
Iy = np.zeros_like(rotated_image)
Ix[:, :-1] = rotated_image[:, 1:] - rotated_image[:, :-1]
Iy[:-1, :] = rotated_image[1:, :] - rotated_image[:-1, :]

# Compute products of derivatives for structure tensor
Ix2 = Ix**2
Iy2 = Iy**2
Ixy = Ix * Iy

def structure_tensor(x, y, window_size=5):
    """Compute structure tensor M and its eigen decomposition at (x,y)"""
    half = window_size // 2
    # Extract local window
    Ix2_win = Ix2[y - half:y + half + 1, x - half:x + half + 1]
    Iy2_win = Iy2[y - half:y + half + 1, x - half:x + half + 1]
    Ixy_win = Ixy[y - half:y + half + 1, x - half:x + half + 1]

    # Sum over window to form M
    M = np.array([
        [Ix2_win.sum(), Ixy_win.sum()],
        [Ixy_win.sum(), Iy2_win.sum()]
    ])

    # Compute eigenvalues and eigenvectors
    eigenvals, eigenvecs = np.linalg.eigh(M)
    return M, eigenvals, eigenvecs

# Define points to analyze: flat, edge, corner
points = {
    'Flat region': (20, 20),
    'Edge': (57, 30),
    'Corner': (22, 50)
}

print("Analyzing structure tensor at selected points:\n")
results = {}

for name, (x, y) in points.items():
    M, eigenvals, eigenvecs = structure_tensor(x, y)
    results[name] = (eigenvals, eigenvecs)
    print(f"{name} at ({x},{y}):")
    print(f"Structure tensor M:\n{M}")
    print(f"Eigenvalues: {eigenvals}")
    print(f"Eigenvectors:\n{eigenvecs}\n")

# Visualization
plt.figure(figsize=(6,6))
plt.imshow(rotated_image, cmap='gray')
plt.title("Rotated Square with Eigenvectors at Selected Points")

colors = {'Flat region': 'blue', 'Edge': 'green', 'Corner': 'red'}

for name, (x, y) in points.items():
    eigenvals, eigenvecs = results[name]
    # Take eigenvector corresponding to largest eigenvalue
    max_eigvec = eigenvecs[:, np.argmax(eigenvals)]
    # Scale eigenvector for visualization
    vec_scale = 10
    plt.scatter(x, y, color=colors[name], label=name, s=100)
    plt.quiver(x, y,
               max_eigvec[0]*vec_scale,
               max_eigvec[1]*vec_scale,
               color=colors[name], angles='xy', scale_units='xy', scale=1)

plt.legend()
plt.axis('off')
plt.show()
